In [1]:
import json, pandas as pd
from pathlib import Path

In [2]:
log_path = Path("../logs/autorizador/autorizador.log")

events = []

with log_path.open(encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

current_event = {}
for ln in lines:
    # Detectar request a validate-signature
    if "Incoming request: POST /validate-signature" in ln:
        current_event = {"test1_firma_valida": None, "test2_tampered_payload": False, "test3_tampered_firma": None, "case": None}

    if "Payload:" in ln and current_event:
        txt = ln.split("Payload:",1)[1].strip()
        safe = (txt.replace("'", '"')
                   .replace("True","true")
                   .replace("False","false")
                   .replace("None","null"))
        try:
            payload = json.loads(safe)
        except Exception:
            payload = {}
        direccion = (payload.get("payload") or {}).get("direccion","")
        if "ALTERADA" in direccion.upper():
            current_event["test2_tampered_payload"] = True

    if "Response JSON:" in ln and current_event:
        txt = ln.split("Response JSON:",1)[1].strip()
        safe = (txt.replace("'", '"')
                   .replace("True","true")
                   .replace("False","false")
                   .replace("None","null"))
        try:
            resp = json.loads(safe)
        except Exception:
            resp = {}
        current_event["test1_firma_valida"] = resp.get("firma_valida")

        # Clasificar caso
        if current_event["test1_firma_valida"]:
            current_event["case"] = "firma_valida"
            expected = True
        else:
            if current_event["test2_tampered_payload"]:
                current_event["case"] = "tampered_payload"
            else:
                current_event["case"] = "tampered_firma"
                current_event["test3_tampered_firma"] = True
            expected = False

        current_event["test_ok"] = (current_event["test1_firma_valida"] == expected)
        events.append(current_event)
        current_event = {}

In [9]:
# DataFrame con resultados
df = pd.DataFrame(events)
display(df)

# resumen
firma_valida_total = sum(1 for e in events if e["case"] == "firma_valida")
firma_valida_passed = sum(1 for e in events if e["case"] == "firma_valida" and e["test_ok"])
print(f"Valid signature -> Total: {firma_valida_total}, Passed: {firma_valida_passed}, Success rate: {(firma_valida_passed/firma_valida_total*100 if firma_valida_total else 0):.2f}%")

tampered_payload_total = sum(1 for e in events if e["case"] == "tampered_payload")
tampered_payload_passed = sum(1 for e in events if e["case"] == "tampered_payload" and e["test_ok"])
print(f"Tampered payload -> Total: {tampered_payload_total}, Passed: {tampered_payload_passed}, Success rate: {(tampered_payload_passed/tampered_payload_total*100 if tampered_payload_total else 0):.2f}%")

tampered_firma_total = sum(1 for e in events if e["case"] == "tampered_firma")
tampered_firma_passed = sum(1 for e in events if e["case"] == "tampered_firma" and e["test_ok"])
print(f"Tampered signature -> Total: {tampered_firma_total}, Passed: {tampered_firma_passed}, Success rate: {(tampered_firma_passed/tampered_firma_total*100 if tampered_firma_total else 0):.2f}%")

,test1_firma_valida,test2_tampered_payload,test3_tampered_firma,case,test_ok
0,True,False,None,firma_valida,True
1,False,True,None,tampered_payload,True
2,False,False,True,tampered_firma,True
3,True,False,None,firma_valida,True
4,False,True,None,tampered_payload,True
...,...,...,...,...,...
356,True,False,None,firma_valida,True
357,True,False,None,firma_valida,True
358,True,False,None,firma_valida,True
359,False,True,None,tampered_payload,True


Valid signature -> Total: 169, Passed: 169, Success rate: 100.00%
Tampered payload -> Total: 96, Passed: 96, Success rate: 100.00%
Tampered signature -> Total: 96, Passed: 96, Success rate: 100.00%


In [4]:
# Resumen por caso
summary = (
    df.groupby("case")["test_ok"]
      .agg(total="count", passed="sum")
      .reset_index()
)
summary["pass_rate_%"] = (summary["passed"] / summary["total"] * 100).round(2)
display(summary)

,case,total,passed,pass_rate_%
0,firma_valida,169,169,100.0
1,tampered_firma,96,96,100.0
2,tampered_payload,96,96,100.0
